In [ ]:
# Create synaptograms from an input json file 

In [ ]:
import numpy as np 
from at_synapse_detection import synaptogram
from at_synapse_detection.render_module import RenderModule
import renderapi
import matplotlib.pyplot as plt
from at_synapse_detection import dataAccess as da
from at_synapse_detection import SynapseDetection as syn 
import os
import json 
from at_synapse_detection import evaluate_synapse_detection as esd
from at_synapse_detection.AnnotationJsonSchema import AnnotationFile, NumpyArray
import importlib
from at_synapse_detection import processDetections as pd


In [ ]:
data = json.load(open('../data/M247514_Rorb_1/Site3Align2/json_annotations/m247514_Site3Annotation_MN_global_v2.json'))

In [ ]:
listOfSynapses = data['area_lists']

In [ ]:
synapse = listOfSynapses[0]

In [ ]:
synapse['oid']

In [ ]:
win_xy = 4
win_z = 1

In [ ]:
bbox = synaptogram.getAnnotationBoundingBox2(synapse)
bbox = synaptogram.transformSynapseCoordinates(bbox)
expandedBox = synaptogram.expandBoundingBox(bbox, win_xy, win_z)

In [ ]:
synapseOutlinesDict = synaptogram.getAnnotationOutlines(synapse)
synapseOutlinesDict = synaptogram.transformSynapseOutlinesDict(synapseOutlinesDict)

In [ ]:
filepath = '/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/';
stackList = ['results', 'PSD95', 'synapsin', 'VGlut1', 'GluN1', 'GABA', 'Gephyrin']

In [ ]:
showProb = True
#filename = 'foo.png'
filename = os.path.join(filepath, '{}.png'.format(synapse['oid']))

In [ ]:
img = synaptogram.getSynaptogramFromFile(bbox, win_xy, win_z, stackList, showProb, filepath);

In [ ]:
%matplotlib notebook

plt.imshow(img, cmap='gray')
plt.colorbar()

In [ ]:
textXOffset = 1
textYOffset = 1

In [ ]:
synaptogram.plotOutlinesOnImg(img, synapseOutlinesDict, expandedBox, filename, stackList, 0, 5)
#plotOutlinesOnImg(img, synapseOutlinesDict, expandedBox, filename, stackList, 0, 5)

In [ ]:
metadataFN = '../data/M247514_Rorb_1/Site3Align2/site3_metadata.json'
metadata = syn.loadMetadata(metadataFN)

datalocation = metadata['datalocation']
queryFN = metadata['querylocation']

# List of Queries
listOfQueries = syn.loadQueriesJSON(queryFN)


In [ ]:
## RUNME 

In [ ]:
args = {
    "EM_annotation_json":"../data/M247514_Rorb_1/Site3Align2/json_annotations/m247514_Site3Annotation_MN_global_v2.json",
    "LM_annotation_json":"../data/M247514_Rorb_1/Site3Align2/results/resultVol9.json",
    "EM_metadata_csv":"../data/M247514_Rorb_1/Site3Align2/MNSite3Synaptograms_v2.csv",
    "LM_metadata_file":"../data/M247514_Rorb_1/Site3Align2/site3_metadata.json",
    "EM_inclass_column":"glutsynapse",
    "EM_not_synapse_column":"ConsensusNotSynapse",
    "output_json":"../data/M247514_Rorb_1/Site3Align2/results/Anish_evaluation_output.json"
    }

In [ ]:
# COMBINE QUERIES

In [ ]:
print(listOfQueries[5])
args['LM_annotation_json'] = "../data/M247514_Rorb_1/Site3Align2/results_07/resultVol5.json"
queryresult1 = pd.evalsyndetections(args)
missedAnnoIds_q2 = pd.getMissedAnnoIds(queryresult1['missed_annotations'])
len(missedAnnoIds_q2)

In [ ]:
print(listOfQueries[6])
args['LM_annotation_json'] = "../data/M247514_Rorb_1/Site3Align2/results_07/resultVol6.json"
queryresult2 = pd.evalsyndetections(args)
missedAnnoIds_q3 = pd.getMissedAnnoIds(queryresult2['missed_annotations'])
len(missedAnnoIds_q3)

In [ ]:
def combineResultVolumes(listOfQueryNumbers, listOfThresholds, args): 

    resultVolList = [] 
    for n, queryNum in enumerate(listOfQueryNumbers): 
    
        fn = os.path.join(metadata['datalocation'], 'resultVol')
        fn = fn + str(queryNum) + '.npy'
        resultVol_n = np.load(fn)
        print(fn)

        resultVol_n = resultVol_n > listOfThresholds[n]
        resultVolList.append(resultVol_n)

    resultVol = resultVolList[0]
    combinedQNum = str(listOfQueryNumbers[0]) 
    for volItr in range(1, len(resultVolList)): 
        resultVol = resultVol + resultVolList[volItr]
        combinedQNum = combinedQNum + str(listOfQueryNumbers[volItr])
        
    combinedQNum = combinedQNum + str(0) + str(0) 
    
    pd.probMapToJSON(resultVol, metadata, combinedQNum)
    
    fn = "../data/M247514_Rorb_1/Site3Align2/results/resultVol" + combinedQNum + ".json"
    
    args['LM_annotation_json'] = fn
    queryresult = pd.evalsyndetections(args)
    missedAnnoIds = pd.getMissedAnnoIds(queryresult['missed_annotations'])
    len(missedAnnoIds)

    return queryresult
    

In [ ]:
listOfQueries[4]

In [ ]:
listOfQueryNumbers = [  3,   4,   6,  11,   15,  19,  20,  21]
listOfThresholds   = [0.7, 0.7, 0.7, 0.7,  0.7, 0.7, 0.7, 0.7]

In [ ]:
queryresult = combineResultVolumes(listOfQueryNumbers, listOfThresholds, args)

In [ ]:
listOfQueryNumbers = [0]
listOfThresholds   = [0.7]
queryresult = combineResultVolumes(listOfQueryNumbers, listOfThresholds, args)

In [ ]:
listOfQueryNumbersPSD = [0]
listOfThresholdsPSD   = [0.9]
queryresultPSD = combineResultVolumes(listOfQueryNumbersPSD, listOfThresholdsPSD, args)

In [ ]:
missedAnnoIdsPSD = pd.getMissedAnnoIds(queryresultPSD['missed_annotations'])


In [ ]:
missedAnnoIds = pd.getMissedAnnoIds(queryresult['missed_annotations'])

In [ ]:
len(missedAnnoIds)

In [ ]:
itr = 0 

old_missed_annotations =  queryresult['missed_annotations']
missed_annotations = []
for n, nID in enumerate(missedAnnoIds): 
    if nID in missedAnnoIdsPSD: 
        foo = 1 
    else: 
        missed_annotations.append(old_missed_annotations[n])
        itr = 1 + itr
        
print(itr)
        

In [ ]:
len(missed_annotations)

In [ ]:
em_edge = queryresult['EM_edge']
np.sum(em_edge)

In [ ]:
# GABA SYNAPSES 

In [ ]:
print(listOfQueries[23])
args['LM_annotation_json'] = "../data/M247514_Rorb_1/Site3Align2/results/resultVol5.json"
queryresult1 = pd.evalGABAsyndetections(args)
missedAnnoIds_q2 = pd.getMissedAnnoIds(queryresult1['missed_annotations'])
len(missedAnnoIds_q2)

In [ ]:
queryresult1.keys()

In [ ]:
good_annotations = (queryresult1['good_annotations'])


In [ ]:
ganno[1:10]

In [ ]:
missedAnnoIds_q2

In [ ]:
win_xy = 4
win_z = 1
filepath = '/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/';
stackList = ['results', 'PSD95', 'synapsin', 'VGlut1', 'GluN1', 'GABA', 'Gephyrin']
showProb = True
textXOffset = 0
textYOffset = 5



In [ ]:
for counter, synapse in enumerate(good_annotations): 
    
#     if (LM_per_EM[counter] > 0 and EM_edge[counter]==False): 
    bbox = synaptogram.getAnnotationBoundingBox2(synapse)
    bbox = synaptogram.transformSynapseCoordinates(bbox)
    expandedBox = synaptogram.expandBoundingBox(bbox, win_xy, win_z)

    synapseOutlinesDict = synaptogram.getAnnotationOutlines(synapse)
    synapseOutlinesDict = synaptogram.transformSynapseOutlinesDict(synapseOutlinesDict)

    filename = os.path.join(filepath, 'synaptograms/good', '{}.png'.format(synapse['oid']))

    img = synaptogram.getSynaptogramFromFile(bbox, win_xy, win_z, stackList, showProb, filepath);
    synaptogram.plotOutlinesOnImg(img, synapseOutlinesDict, expandedBox, filename, stackList, textXOffset, textYOffset)
    print(filename)

In [ ]:
for counter, synapse in enumerate(missed_annotations): 
    
    #if synapse['oid'] in missed_anno: 
    bbox = synaptogram.getAnnotationBoundingBox2(synapse)
    bbox = synaptogram.transformSynapseCoordinates(bbox)
    expandedBox = synaptogram.expandBoundingBox(bbox, win_xy, win_z)

    synapseOutlinesDict = synaptogram.getAnnotationOutlines(synapse)
    synapseOutlinesDict = synaptogram.transformSynapseOutlinesDict(synapseOutlinesDict)

    filename = os.path.join(filepath, 'synaptograms/missed', '{}.png'.format(synapse['oid']))

    img = synaptogram.getSynaptogramFromFile(bbox, win_xy, win_z, stackList, showProb, filepath);
    synaptogram.plotOutlinesOnImg(img, synapseOutlinesDict, expandedBox, filename, stackList, textXOffset, textYOffset)
    print(filename)

In [ ]:
#false postives 

In [ ]:
false_positives = queryresult['false_positives']

In [ ]:
false_positives[0]

In [ ]:
for counter, synapse in enumerate(false_positives): 

    bbox = synaptogram.getAnnotationBoundingBox2(synapse)
    bbox = synaptogram.transformSynapseCoordinates(bbox)
    expandedBox = synaptogram.expandBoundingBox(bbox, win_xy, win_z)

    synapseOutlinesDict = synaptogram.getAnnotationOutlines(synapse)
    synapseOutlinesDict = synaptogram.transformSynapseOutlinesDict(synapseOutlinesDict)

    filename = os.path.join(filepath, 'synaptograms/false_positives', '{}.png'.format(synapse['oid']))

    img = synaptogram.getSynaptogramFromFile(bbox, win_xy, win_z, stackList, showProb, filepath);
    synaptogram.plotOutlinesOnImg(img, synapseOutlinesDict, expandedBox, filename, stackList, textXOffset, textYOffset)
    print(filename)

In [ ]:
for n in range(1, 2): 
    print(n) 

In [ ]:
vol = da.imreadtiffseries('/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/GABA')
img = vol[:, :, 0]

In [ ]:
scale_param = np.sqrt(0.5 * np.mean(np.power(img.flatten(), 2)));

In [ ]:
x = img

In [ ]:
probimg = 1-np.exp((-np.power(x, 2))/(2*np.power(scale_param, 2)));

In [ ]:
probimg

In [ ]:
from scipy.stats import norm,rayleigh
from skimage import measure
import scipy

In [ ]:
aLOC, bSCALE = scipy.stats.rayleigh.fit(img.flatten(), floc=0)

In [ ]:
aLOC, bSCALE

In [ ]:
probimg = rayleigh.cdf(img, loc=aLOC, scale=bSCALE)

In [ ]:
labelVol = measure.label(probimg > 0.9, 8)
stats = measure.regionprops(labelVol)
print(len(stats)) 

In [ ]:
%matplotlib notebook

plt.imshow(probimg, cmap='gray')

In [ ]:
args